## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import gmaps
import json
import numpy as np
import pandas as pd
import requests
import urllib

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

csv_file_name = "WeatherPy_Vacation.csv"

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Before going further, let's convert temperatures from Kelvin to Celsius
city_data_df["Max Temp"] = city_data_df["Max Temp"] - 273.15
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lorengau,PG,-2.0226,147.2712,26.97,83,100,4.52,overcast clouds
1,1,Lumberton,US,34.6182,-79.0086,33.21,39,48,5.70,scattered clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,-1.09,69,40,8.75,scattered clouds
3,3,Kruisfontein,ZA,-34.0033,24.7314,15.49,74,14,9.41,few clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,26.98,88,71,1.79,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input("Please enter your desired minimum temperature: "))
max_temp = int(input("Please enter your desired maximum temperature: "))

Please enter your desired minimum temperature: 25
Please enter your desired maximum temperature: 30


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a
# new DataFrame using the loc method.
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] > min_temp) &
                               (city_data_df["Max Temp"] < max_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
len(filtered_df.loc[filtered_df["Max Temp"] == np.nan])

0

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_df.loc[filtered_df["Max Temp"] != np.nan]

In [43]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[
    ["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]
].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lorengau,PG,26.97,overcast clouds,-2.0226,147.2712,
4,Puerto Ayora,EC,26.98,broken clouds,-0.7393,-90.3518,
7,Maceió,BR,26.69,few clouds,-9.6658,-35.7353,
29,Hilo,US,27.11,scattered clouds,19.7297,-155.0900,
36,Hong Kong,HK,29.98,overcast clouds,22.2855,114.1577,
38,Kavieng,PG,27.91,overcast clouds,-2.5744,150.7967,
48,Maţāy,EG,26.80,clear sky,28.4190,30.7792,
54,Bonthe,SL,27.96,broken clouds,7.5264,-12.5050,
65,Piracuruca,BR,27.15,broken clouds,-3.9281,-41.7092,
77,Sorong,ID,25.67,scattered clouds,-0.8833,131.2500,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_maps_url = (
        "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
        "radius=" + str(params["radius"]) + "&"
        "type=" + params["type"] + "&"
        "key=" + g_key + "&"
        "location=")

    # 6e. Make request and retrieve the JSON data from the search.
    full_url = base_maps_url + str(lat) + "," + str(lng)
    with urllib.request.urlopen(full_url) as url:
        data = json.loads(url.read().decode())
        # 6f. Get the first hotel from the results and store the name, if a hotel
        # isn't found skip the city.
        if len(data["results"]) > 0:
            hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]

In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lorengau,PG,26.97,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
4,Puerto Ayora,EC,26.98,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Maceió,BR,26.69,few clouds,-9.6658,-35.7353,Poutur Pousada
29,Hilo,US,27.11,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
36,Hong Kong,HK,29.98,overcast clouds,22.2855,114.1577,Mini Hotel Central
38,Kavieng,PG,27.91,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
48,Maţāy,EG,26.80,clear sky,28.4190,30.7792,Nag Aespen
54,Bonthe,SL,27.96,broken clouds,7.5264,-12.5050,Bonthe Holiday Village
65,Piracuruca,BR,27.15,broken clouds,-3.9281,-41.7092,Pousada Piracuruca
77,Sorong,ID,25.67,scattered clouds,-0.8833,131.2500,waigo splash hotel


In [14]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(csv_file_name, index_label="City_ID")

In [9]:
# Now reimport the CSV so that we don't need to repeat the API calls
clean_hotel_df = pd.read_csv(csv_file_name)

In [17]:
# 9. Using the template add city name, the country code, the weather
# description and maximum temperature for the city.
info_box_template = (
    "<b>Hotel Name</b><br/>{Hotel Name}<br/>"
    "<b>City</b><br/>{City}<br/>"
    "<b>Country</b><br/>{Country}<br/>"
    "<b>Current Weather</b><br/>{Current Description}<br/>"
)

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))